In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
tf.keras.backend.clear_session()

In [8]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [9]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [10]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Consumo de Cimento (t)
0,2008,6.535602e+07,2.793553e+06,17.127903,5.260918e+07,0.752766,0.787367,0.732454,5.483335e+08,2.546644e+09,...,0.738916,8.063005,31.686523,993.754162,1421.360784,26.585520,19.247754,68.306560,1.265376,994.957000
1,2009,6.593967e+07,2.815133e+06,17.131657,5.267158e+07,0.753704,0.788247,0.735581,5.302764e+08,2.679945e+09,...,0.738104,8.095019,26.992644,992.577520,971.624734,26.514656,18.393406,65.147890,1.139212,998.465000
2,2010,6.789156e+07,2.861375e+06,17.294037,5.339572e+07,0.754460,0.788564,0.738478,5.046425e+08,2.891930e+09,...,0.737478,8.057848,25.116066,989.658472,896.219335,26.516966,17.684947,62.460338,1.378783,1099.340000
3,2011,6.985811e+07,2.887266e+06,17.471861,5.415240e+07,0.755039,0.788328,0.741142,4.854622e+08,3.091988e+09,...,0.737032,8.047069,28.216671,987.048009,916.320612,26.594241,18.048644,63.561988,1.548197,1204.074000
4,2012,7.159016e+07,2.905726e+06,17.591515,5.472408e+07,0.755027,0.787468,0.742721,4.867661e+08,3.189506e+09,...,0.736572,6.695389,27.946087,986.033104,884.118586,26.504045,18.419653,65.231377,1.625213,1381.539000
5,2013,7.314074e+07,2.912179e+06,17.684470,5.519726e+07,0.756113,0.786174,0.746782,5.225848e+08,3.093872e+09,...,0.736953,7.068881,31.557800,985.745432,817.482501,26.560721,18.794569,66.303790,1.668395,1618.657000
6,2014,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.755956,0.784438,0.748013,5.676413e+08,2.779340e+09,...,0.736866,7.495401,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375,1411.234429
7,2015,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.755452,0.782141,0.749014,5.179392e+08,2.561731e+09,...,0.736688,7.838256,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487,1241.100343
8,2016,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.758777,0.788475,0.756866,4.524144e+08,2.516859e+09,...,0.732916,7.730867,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712,1083.128229
9,2017,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.757848,0.785609,0.756036,4.213689e+08,2.497710e+09,...,0.733636,7.251143,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939,1220.058000


In [11]:
input_data = data.iloc[:-2, 1:-1]
input_data = input_data.drop('Mato Grosso - Produção de Cimento (t)',axis=1)
input_data

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,6.535602e+07,2.793553e+06,17.127903,5.260918e+07,0.752766,0.787367,0.732454,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,0.738916,8.063005,31.686523,993.754162,1421.360784,26.585520,19.247754,68.306560,1.265376
1,6.593967e+07,2.815133e+06,17.131657,5.267158e+07,0.753704,0.788247,0.735581,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,0.738104,8.095019,26.992644,992.577520,971.624734,26.514656,18.393406,65.147890,1.139212
2,6.789156e+07,2.861375e+06,17.294037,5.339572e+07,0.754460,0.788564,0.738478,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,0.737478,8.057848,25.116066,989.658472,896.219335,26.516966,17.684947,62.460338,1.378783
3,6.985811e+07,2.887266e+06,17.471861,5.415240e+07,0.755039,0.788328,0.741142,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,0.737032,8.047069,28.216671,987.048009,916.320612,26.594241,18.048644,63.561988,1.548197
4,7.159016e+07,2.905726e+06,17.591515,5.472408e+07,0.755027,0.787468,0.742721,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,0.736572,6.695389,27.946087,986.033104,884.118586,26.504045,18.419653,65.231377,1.625213
5,7.314074e+07,2.912179e+06,17.684470,5.519726e+07,0.756113,0.786174,0.746782,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,0.736953,7.068881,31.557800,985.745432,817.482501,26.560721,18.794569,66.303790,1.668395
6,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.755956,0.784438,0.748013,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,0.736866,7.495401,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375
7,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.755452,0.782141,0.749014,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,0.736688,7.838256,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487
8,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.758777,0.788475,0.756866,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,0.732916,7.730867,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712
9,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.757848,0.785609,0.756036,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,0.733636,7.251143,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939


In [12]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      998.465000
1     1099.340000
2     1204.074000
3     1381.539000
4     1618.657000
5     1411.234429
6     1241.100343
7     1083.128229
8     1220.058000
9     1316.386000
10    1446.064000
11    1555.266000
12    1788.146000
13    1947.017000
14            NaN
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [13]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.740588,-1.627429,-1.768928,-1.746450,-1.736363,0.561629,-1.727009,1.204752,-0.990234,0.166091,...,0.984445,1.450893,1.102745,1.214956,2.196775,2.448959,-0.498590,1.635424,1.610412,-1.533856
1,-1.598275,-1.055001,-1.755010,-1.698578,-1.152572,0.837117,-1.343029,0.820657,-0.546688,0.428911,...,-0.051710,1.051022,1.170358,-0.661837,1.783361,0.503680,-0.709406,-0.183740,0.176596,-2.299018
2,-1.122342,0.171630,-1.152996,-1.143071,-0.682504,0.936364,-0.987463,0.275392,0.158672,0.847995,...,-0.115145,0.743048,1.091854,-1.412165,0.757751,0.177523,-0.702533,-1.692261,-1.043365,-0.846063
3,-0.642835,0.858410,-0.493731,-0.562596,-0.322949,0.862562,-0.660386,-0.132596,0.824346,1.230291,...,0.633403,0.523822,1.069089,-0.172424,-0.159437,0.264469,-0.472647,-0.917841,-0.543293,0.181401
4,-0.220508,1.348072,-0.050124,-0.124039,-0.329912,0.593304,-0.466463,-0.104860,1.148829,1.382743,...,-0.646476,0.297449,-1.785600,-0.280614,-0.516025,0.125183,-0.740974,-0.127851,0.214493,0.648489
5,0.157572,1.519248,0.294500,0.238952,0.345134,0.188062,0.031981,0.657046,0.830615,1.112297,...,-0.757614,0.484935,-0.996801,1.163487,-0.617099,-0.163043,-0.572364,0.670458,0.701294,0.910382
6,0.490699,1.147933,0.594458,0.545796,0.247355,-0.355312,0.183203,1.615453,-0.215960,0.282052,...,0.345772,0.442014,-0.096011,1.901614,-0.674184,-0.547688,0.117409,1.103701,0.676713,1.037619
7,0.738027,0.345838,0.740713,0.731890,-0.065842,-1.074422,0.306105,0.558230,-0.940033,-0.632425,...,1.298413,0.354512,0.628086,0.952359,-0.774374,-0.969163,-0.034864,0.872930,0.613224,0.923072
8,0.931149,-0.452241,0.886969,0.897373,2.001700,0.908478,1.270060,-0.835563,-1.089341,-1.093812,...,1.596124,-1.501484,0.401284,-0.309032,-0.829906,-1.336391,-0.314910,0.475739,0.482521,0.603000
9,0.998790,-0.757316,0.876303,0.933102,1.423819,0.011185,1.168125,-1.495939,-1.153056,-1.335633,...,-0.048438,-1.147363,-0.611873,-1.048571,-0.860916,-1.224770,-0.073054,0.221037,0.236176,0.252619


In [44]:
reshaped_train_input = dfToInputRNN(train_input)

In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      998.465000
1     1099.340000
2     1204.074000
3     1381.539000
4     1618.657000
5     1411.234429
6     1241.100343
7     1083.128229
8     1220.058000
9     1316.386000
10    1446.064000
11    1555.266000
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [70]:
# Alvo para validação do treinamento
validation_target = dfToInputRNN(train_input.iloc[-1:])
validation_target

array([[[ 0.96688501, -0.687973  ,  0.88887289,  0.93415987,
         -0.46474288, -2.387435  ,  1.06750828, -0.96642323,
          2.12956681, -0.97721778,  0.83995693,  2.07628186,
         -0.50761395,  1.38827342, -0.51218717, -0.24869819,
          1.75442212,  1.9436176 , -2.48498166,  1.27843653,
          1.00480026,  0.51934554,  0.6157454 ,  0.96370542,
          2.12840428,  1.83889023,  1.30109117,  1.4565741 ,
          1.7919157 ,  1.1812493 ,  0.71605861, -2.07993955,
          1.76021533,  1.37062734,  1.40267921,  0.88563323,
         -2.05700225, -2.27654367, -2.51822607, -0.4355815 ,
         -1.00746095,  0.41200983, -0.35697472,  2.29949195,
          1.38196517,  1.94744017,  2.02810693,  1.77994091,
         -2.19986403,  2.53262347,  1.3430515 ,  0.1196425 ,
          1.64452866,  1.63649261,  1.77741481,  1.99205361,
          1.55717148,  1.52538793,  1.93204449,  3.14115558,
         -0.69303478,  2.4022204 , -1.40094792, -1.72331776,
         -1.57993601, -1

In [17]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.765494,-0.551552,0.800035,0.794933,-1.569273,-2.551374,0.831359,1.108405,2.635823,-0.189959,...,-2.260447,-1.55722,-1.265069,-0.730185,0.611707,1.620843,2.316005,-1.810579,-2.274914,0.168633


In [18]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.76549398, -0.55155174,  0.80003549,  0.79493323,
         -1.56927324, -2.55137369,  0.83135931,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
         -1.55721996, -1

In [19]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1788.146
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [125]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [126]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [127]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3050311493, 2968501436, 1418814030, 4089376668, 2439445352, 1131422551, 2573205647, 2918174483, 1865935635, 3625869072, 854252365, 2498152598, 1258608632, 3290642454, 938766181, 3698361630, 2600899189, 3814898099, 3257957348, 483567422, 1971176289, 1027655537, 815198344, 1239696514, 3551662717, 1912729765, 429568528, 1440794549, 2071728988, 2509004518, 2534916577, 3424493299, 3564310478, 668180627, 3993022314, 521626944, 198380605, 988482643, 227118609, 3320062527, 1570535935, 2779117499, 2790171236, 2678496919, 381822203, 741727640, 3234220650, 2522422278, 3818501244, 1356414469, 513968109, 3899570726, 3127332266, 2235074716, 1785067778, 4116321504, 3568479155, 3512464614, 4203981782, 3382120877, 779221022, 4230444604, 3923784056, 2941789065, 1279189219, 258250752, 4065089443, 1692672840, 3868277992, 3132302341, 1182097169, 2710569777, 4114222919, 377397469, 1570560036, 3925544704, 3746175973, 575005430, 2229699923, 3476314784, 1937424627, 690757245, 3478280391, 3757503300, 342212140

loss: 1421.3238525390625


Step: 87 ___________________________________________
loss: 1000.2608032226562


Step: 88 ___________________________________________
loss: 1686.1776123046875


Step: 89 ___________________________________________
loss: 1323.9991455078125


Step: 90 ___________________________________________
loss: 1955.7564697265625


Step: 91 ___________________________________________
loss: 932.1648559570312


Step: 92 ___________________________________________
loss: 1217.4500732421875


Step: 93 ___________________________________________
loss: 1660.3228759765625


Step: 94 ___________________________________________
loss: 817.558837890625


Step: 95 ___________________________________________
loss: 528.0961303710938


Step: 96 ___________________________________________
loss: 856.0685424804688


Step: 97 ___________________________________________
loss: 785.3892211914062


Step: 98 ___________________________________________
loss: 1002.5090942382812


Step: 99 __________

In [128]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 1718375.8750
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 1680074.3750
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 1489179.1250
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 367509.5312
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 75635.7266
Epoch 6/10000
1/1 [==============================] - 0s 8ms/step - loss: 41580.1836
Epoch 7/10000
1/1 [==============================] - 0s 5ms/step - loss: 42716.8984
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 52124.6094
Epoch 9/10000
1/1 [==============================] - 0s 9ms/step - loss: 33652.3164
Epoch 10/10000
1/1 [==============================] - 0s 5ms/step - loss: 34839.6836
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 34508.2461
Epoch 12/10000
1/1 [==============================] - 0s 7ms/step - 

1/1 [==============================] - 0s 6ms/step - loss: 8805.4326
Epoch 98/10000
1/1 [==============================] - 0s 7ms/step - loss: 27088.1875
Epoch 99/10000
1/1 [==============================] - 0s 7ms/step - loss: 6665.5952
Epoch 100/10000
1/1 [==============================] - 0s 7ms/step - loss: 9574.1084
Epoch 101/10000
1/1 [==============================] - 0s 6ms/step - loss: 10849.8906
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 11654.8086
Epoch 103/10000
1/1 [==============================] - 0s 7ms/step - loss: 7698.7915
Epoch 104/10000
1/1 [==============================] - 0s 6ms/step - loss: 7460.2212
Epoch 105/10000
1/1 [==============================] - 0s 7ms/step - loss: 9814.4795
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 12171.5391
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 9733.4756
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - loss: 7404

1/1 [==============================] - 0s 5ms/step - loss: 4826.0952
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 8519.3408
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 8495.1611
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 17352.8496
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 7544.9868
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 10298.0342
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 4304.5923
Epoch 200/10000
1/1 [==============================] - 0s 5ms/step - loss: 6286.2847
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 8637.2998
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 5155.4243
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - loss: 4747.0703
Epoch 204/10000
1/1 [==============================] - 0s 5ms/step - loss: 6549

1/1 [==============================] - 0s 5ms/step - loss: 10140.0381
Epoch 290/10000
1/1 [==============================] - 0s 6ms/step - loss: 7082.6401
Epoch 291/10000
1/1 [==============================] - 0s 7ms/step - loss: 6940.5820
Epoch 292/10000
1/1 [==============================] - 0s 6ms/step - loss: 10635.3154
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 11610.2236
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 13701.6299
Epoch 295/10000
1/1 [==============================] - 0s 5ms/step - loss: 5543.2793
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 12364.4248
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 8864.8896
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 10577.6670
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 4726.6860
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 

1/1 [==============================] - 0s 4ms/step - loss: 8892.0234
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 8931.4775
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 8724.2441
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 3303.4463
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 4879.6279
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 8809.1211
Epoch 391/10000
1/1 [==============================] - 0s 5ms/step - loss: 8216.9688
Epoch 392/10000
1/1 [==============================] - 0s 5ms/step - loss: 5474.1401
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 5916.9468
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 9169.9092
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 7451.4409
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 4456.2

...

1/1 [==============================] - 0s 4ms/step - loss: 5912.2075
Epoch 1247/10000
1/1 [==============================] - 0s 5ms/step - loss: 6383.2476
Epoch 1248/10000
1/1 [==============================] - 0s 4ms/step - loss: 3515.5662
Epoch 1249/10000
1/1 [==============================] - 0s 4ms/step - loss: 6219.2466
Epoch 1250/10000
1/1 [==============================] - 0s 5ms/step - loss: 6184.7368
Epoch 1251/10000
1/1 [==============================] - 0s 4ms/step - loss: 7609.9062
Epoch 1252/10000
1/1 [==============================] - 0s 4ms/step - loss: 3580.3105
Epoch 1253/10000
1/1 [==============================] - 0s 4ms/step - loss: 3395.0449
Epoch 1254/10000
1/1 [==============================] - 0s 4ms/step - loss: 6199.6250
Epoch 1255/10000
1/1 [==============================] - 0s 4ms/step - loss: 6253.1431
Epoch 1256/10000
1/1 [==============================] - 0s 5ms/step - loss: 11053.4180
Epoch 1257/10000
1/1 [==============================] - 0s 4ms/step - 

1/1 [==============================] - 0s 6ms/step - loss: 6377.0122
Epoch 1342/10000
1/1 [==============================] - 0s 5ms/step - loss: 3513.7488
Epoch 1343/10000
1/1 [==============================] - 0s 5ms/step - loss: 6093.3789
Epoch 1344/10000
1/1 [==============================] - 0s 5ms/step - loss: 4663.4868
Epoch 1345/10000
1/1 [==============================] - 0s 5ms/step - loss: 2219.1980
Epoch 1346/10000
1/1 [==============================] - 0s 5ms/step - loss: 2042.6289
Epoch 1347/10000
1/1 [==============================] - 0s 5ms/step - loss: 4149.6230
Epoch 1348/10000
1/1 [==============================] - 0s 5ms/step - loss: 5018.9292
Epoch 1349/10000
1/1 [==============================] - 0s 5ms/step - loss: 6555.6548
Epoch 1350/10000
1/1 [==============================] - 0s 6ms/step - loss: 5539.5337
Epoch 1351/10000
1/1 [==============================] - 0s 6ms/step - loss: 2652.2617
Epoch 1352/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 2082.6794
Epoch 1437/10000
1/1 [==============================] - 0s 6ms/step - loss: 6458.7993
Epoch 1438/10000
1/1 [==============================] - 0s 6ms/step - loss: 9291.8594
Epoch 1439/10000
1/1 [==============================] - 0s 6ms/step - loss: 5164.5000
Epoch 1440/10000
1/1 [==============================] - 0s 6ms/step - loss: 8825.8662
Epoch 1441/10000
1/1 [==============================] - 0s 6ms/step - loss: 9401.8662
Epoch 1442/10000
1/1 [==============================] - 0s 5ms/step - loss: 6728.5938
Epoch 1443/10000
1/1 [==============================] - 0s 5ms/step - loss: 7541.4399
Epoch 1444/10000
1/1 [==============================] - 0s 5ms/step - loss: 7490.2930
Epoch 1445/10000
1/1 [==============================] - 0s 4ms/step - loss: 5663.0210
Epoch 1446/10000
1/1 [==============================] - 0s 5ms/step - loss: 8312.9424
Epoch 1447/10000
1/1 [==============================] - 0s 6ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 4429.0376
Epoch 1532/10000
1/1 [==============================] - 0s 7ms/step - loss: 6848.5469
Epoch 1533/10000
1/1 [==============================] - 0s 5ms/step - loss: 6631.8208
Epoch 1534/10000
1/1 [==============================] - 0s 5ms/step - loss: 5622.2153
Epoch 1535/10000
1/1 [==============================] - 0s 5ms/step - loss: 7949.1992
Epoch 1536/10000
1/1 [==============================] - 0s 5ms/step - loss: 3186.3252
Epoch 1537/10000
1/1 [==============================] - 0s 4ms/step - loss: 5937.2290
Epoch 1538/10000
1/1 [==============================] - 0s 5ms/step - loss: 3632.4680
Epoch 1539/10000
1/1 [==============================] - 0s 5ms/step - loss: 5477.6250
Epoch 1540/10000
1/1 [==============================] - 0s 5ms/step - loss: 5535.4741
Epoch 1541/10000
1/1 [==============================] - 0s 5ms/step - loss: 4170.4546
Epoch 1542/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 7ms/step - loss: 4790.9336
Epoch 1627/10000
1/1 [==============================] - 0s 5ms/step - loss: 6001.8784
Epoch 1628/10000
1/1 [==============================] - 0s 5ms/step - loss: 7477.3687
Epoch 1629/10000
1/1 [==============================] - 0s 7ms/step - loss: 3825.2754
Epoch 1630/10000
1/1 [==============================] - 0s 6ms/step - loss: 5956.7778
Epoch 1631/10000
1/1 [==============================] - 0s 4ms/step - loss: 4051.4622
Epoch 1632/10000
1/1 [==============================] - 0s 4ms/step - loss: 4389.9844
Epoch 1633/10000
1/1 [==============================] - 0s 5ms/step - loss: 4312.2231
Epoch 1634/10000
1/1 [==============================] - 0s 4ms/step - loss: 4413.5083
Epoch 1635/10000
1/1 [==============================] - 0s 5ms/step - loss: 4586.1943
Epoch 1636/10000
1/1 [==============================] - 0s 7ms/step - loss: 10399.8672
Epoch 1637/10000
1/1 [==============================] - 0s 7ms/step - 

In [129]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 577ms/step
[1788.146](test_target) - [[1552.651]](prediction) = 235.49499902343746


In [130]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.13169785857722885

In [131]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [132]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.958981,-1.053327,-0.731205,-0.792708,-1.266344,-1.367241,-1.240125,1.148205,-1.122043,-1.121418,...,-1.199016,1.274172,-0.544470,1.358778,1.020728,1.401690,1.413631,1.261025,1.256285,0.043458
1,-0.420657,-0.290573,-0.682732,-0.617899,0.087939,0.370587,0.031362,0.140893,-0.184474,-0.185492,...,-0.049930,-0.105707,1.402573,-0.339841,0.337331,-0.538217,-0.741954,-0.076111,-0.065726,-1.245895
2,1.379638,1.343901,1.413937,1.410608,1.178406,0.996653,1.208763,-1.289098,1.306518,1.306910,...,1.248946,-1.168466,-0.858104,-1.018937,-1.358059,-0.863473,-0.671678,-1.184915,-1.190558,1.202438


train_target:


,Mato Grosso - Consumo de Cimento (t)
0,998.465
1,1099.340
2,1204.074


test_input:


...

,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH,Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Renda,Mato Grosso - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.765494,-0.551552,0.800035,0.794933,-1.569273,-2.551374,0.831359,1.108405,2.635823,-0.189959,...,1.207226,-1.55722,-1.265069,-0.730185,0.611707,1.620843,2.316005,-1.810579,-2.274914,0.168633


test_target:


,Mato Grosso - Consumo de Cimento (t)
12,1788.146


1/1 [==============================] - 1s 583ms/step
Error: 255.77759179687496




[1175.2020263671875,
 1360.8846435546875,
 1348.2445068359375,
 1407.7135009765625,
 1255.148193359375,
 1114.9892578125,
 1155.8017578125,
 1232.102294921875,
 1425.59521484375,
 1532.368408203125]

In [133]:
display(targets)
display(predictions)
display(np.array(targets) - np.array(predictions))

[1381.539,
 1618.657,
 1411.2344285714282,
 1241.1003428571426,
 1083.1282285714285,
 1220.058,
 1316.386,
 1446.064,
 1555.266,
 1788.146]

[1175.2020263671875,
 1360.8846435546875,
 1348.2445068359375,
 1407.7135009765625,
 1255.148193359375,
 1114.9892578125,
 1155.8017578125,
 1232.102294921875,
 1425.59521484375,
 1532.368408203125]

array([ 206.33697363,  257.77235645,   62.98992174, -166.61315812,
       -172.01996479,  105.06874219,  160.58424219,  213.96170508,
        129.67078516,  255.7775918 ])

In [134]:
mae = mean_absolute_error(predictions, targets)
mae

173.07954411272323

In [135]:
porcentage = mae/np.mean(targets)
porcentage

0.12308684829258737